In [22]:
import configparser
import psycopg2
import socket
import sys

In [19]:

config = configparser.ConfigParser()
config.read('../config.ini')
db_endpoint_RDS = config['database']['host']
db_name_RDS = config['database']['database_name']
db_user_RDS = config['database']['username']
db_password_RDS = config['database']['password']
db_port_RDS = int(config['database']['port'])

db_endpoint_RSH = config['redshift']['host']
db_name_RSH = config['redshift']['database_name']
db_user_RSH = config['redshift']['username']
db_password_RSH = config['redshift']['password']
db_port_RSH = int(config['redshift']['port'])

def connect_to_RDS():
    conn_rds = psycopg2.connect(
            host=db_endpoint_RDS,
            port = db_port_RDS,
            database = db_name_RDS,
            user=db_user_RDS,
            password=db_password_RDS
        )
    return conn_rds

def connection_to_RedShift():

    conn_rsh = psycopg2.connect(
            host=db_endpoint_RSH,
            port = db_port_RSH,
            database = db_name_RSH,
            user=db_user_RSH,
            password=db_password_RSH
        )
    
    return conn_rsh


try:    

    conn_rds = connect_to_RDS()
    cur_rds = conn_rds.cursor()
    conn_rsh = connection_to_RedShift()
    cur_rsh = conn_rsh.cursor()


except (socket.timeout, psycopg2.OperationalError) as e:
    if isinstance(e, socket.timeout):
        print("Error: Connection timed out.")
    else:
        print("Error during connection:", e)
    sys.exit(1)  # Terminate the program with a non-zero exit code


try:
    # Obtener los símbolos únicos de tbTradingHistoric
    cur_rds.execute("SELECT DISTINCT \"Symbol\" FROM tbTradingHistoric;")
    symbols_tbTradingHistoric = cur_rds.fetchall()
    print("Symbols fetched from tbTradingHistoric successfully!")

    # Obtener los símbolos únicos de tbDimSymbol
    cur_rsh.execute("SELECT DISTINCT \"Symbol\" FROM tbDimSymbol;")
    symbols_tbdimSymbols = cur_rsh.fetchall()
    print("Symbols fetched from tbDimSymbol successfully!")

    # Lista de comprensión para eliminar los símbolos existentes de symbols_tbTradingHistoric
    symbols_tbTradingHistoric = [symbol for symbol in symbols_tbTradingHistoric if symbol not in symbols_tbdimSymbols]

    # Consulta de inserción
    insert_query = "INSERT INTO tbDimSymbol (Symbol) VALUES (%s)"

    with conn_rsh.cursor() as cur_rsh:
        for symbol in symbols_tbTradingHistoric:
            try:
                cur_rsh.execute(insert_query, symbol)
                print(f"Inserted symbol '{symbol[0]}' into tbDimSymbol successfully!")
            except psycopg2.Error as e:
                print(f"Error occurred during insertion for symbol '{symbol[0]}':", e)
                conn_rsh.rollback()  # Rollback the transaction in case of an error

    conn_rsh.commit()  # Commit all the successful insertions

except (socket.timeout, psycopg2.OperationalError) as e:
    if isinstance(e, socket.timeout):
        print("Error: Connection timed out.")
    else:
        print("Error during connection:", e)
    sys.exit(1)  # Terminate the program with a non-zero exit code
except psycopg2.Error as e:
    print("Error occurred during SQL query:", e)
    conn_rsh.rollback()  # Rollback the transaction in case of an error


conn_rds.close()
conn_rsh.close()

Symbols fetched from tbTradingHistoric successfully!
Symbols fetched from tbDimSymbol successfully!


In [23]:

config = configparser.ConfigParser()
config.read('../config.ini')
db_endpoint_RDS = config['database']['host']
db_name_RDS = config['database']['database_name']
db_user_RDS = config['database']['username']
db_password_RDS = config['database']['password']
db_port_RDS = int(config['database']['port'])

db_endpoint_RSH = config['redshift']['host']
db_name_RSH = config['redshift']['database_name']
db_user_RSH = config['redshift']['username']
db_password_RSH = config['redshift']['password']
db_port_RSH = int(config['redshift']['port'])

def connect_to_RDS():
    conn_rds = psycopg2.connect(
            host=db_endpoint_RDS,
            port = db_port_RDS,
            database = db_name_RDS,
            user=db_user_RDS,
            password=db_password_RDS
        )
    return conn_rds

def connection_to_RedShift():

    conn_rsh = psycopg2.connect(
            host=db_endpoint_RSH,
            port = db_port_RSH,
            database = db_name_RSH,
            user=db_user_RSH,
            password=db_password_RSH
        )
    
    return conn_rsh


try:    

    conn_rds = connect_to_RDS()
    cur_rds = conn_rds.cursor()
    conn_rsh = connection_to_RedShift()
    cur_rsh = conn_rsh.cursor()


except (socket.timeout, psycopg2.OperationalError) as e:
    if isinstance(e, socket.timeout):
        print("Error: Connection timed out.")
    else:
        print("Error during connection:", e)
    sys.exit(1)  # Terminate the program with a non-zero exit code

In [24]:
cur_rds.execute("SELECT DISTINCT \"Symbol\" FROM tbTradingHistoric;")
symbols_tbTradingHistoric = cur_rds.fetchall()
print("Symbols fetched from tbTradingHistoric successfully!")

cur_rds.execute("SELECT DISTINCT \"Symbol\" FROM tbtickers;")
symbols_tbtickers = cur_rds.fetchall()
print("Symbols fetched from tbTradingHistoric successfully!")


cur_rds.execute("SELECT DISTINCT \"Symbol\" FROM tbGIDSDirectory;")
symbols_tbgidsdirectory = cur_rds.fetchall()
print("Symbols fetched from tbTradingHistoric successfully!")

combined_symbols = symbols_tbTradingHistoric + symbols_tbtickers + symbols_tbgidsdirectory

# Obtener los símbolos únicos de tbDimSymbol
cur_rsh.execute("SELECT DISTINCT \"Symbol\" FROM tbDimSymbol;")
symbols_tbdimSymbols = cur_rsh.fetchall()
print("Symbols fetched from tbDimSymbol successfully!")

# Lista de comprensión para eliminar los símbolos existentes de symbols_tbTradingHistoric
combined_symbols = list(set([symbol][0] for symbol in combined_symbols if symbol not in symbols_tbdimSymbols))


Symbols fetched from tbTradingHistoric successfully!
Symbols fetched from tbTradingHistoric successfully!
Symbols fetched from tbTradingHistoric successfully!
Symbols fetched from tbDimSymbol successfully!


In [25]:
from collections import Counter

symbol_counts = Counter(symbol[0] for symbol in combined_symbols)

has_duplicates = any(count > 1 for count in symbol_counts.values())

if has_duplicates:
    print("la lista tiene duplicados")
else:
    print("la lista no contiene duplicados")

la lista no contiene duplicados


In [27]:
def has_nulls(list):
    return any(item is None for item in list)

print(has_nulls(combined_symbols))

False
